# Passos para a realização da classificação

- Carrega o arquivo `fif`(`mne.Ep`) dos dados *filtrados*;
- *Obter a "energia"* do sinal por meio do cálculo `compute_psd`_;
- *Determine o limiar* para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
- Com as listas de pontos isoladas para cada estimulo, aplique uma caraceristica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um *vetor de características*;
- Por fim, realize a *classificação*, que será um *cálculo de voto* simples (maior valor é provavelmente o da frequência evocada).

In [4]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import mne
import warnings

### Single Target

In [5]:
warnings.filterwarnings('ignore')
# mne_d = mne.read_epochs("../datasets/avi/single/filtered-4-14-sub-0.fif")

# event_dict = {'6':0, '6.5':1, '7':2, '7.5':3, '8.2':4, '9.3':5, '10':6}
values = {6: 6, 6.5: 6.5, 7: 7, 7.5: 7.5, 8.2: 8.2, 9.3: 9.3, 10: 10}

labels_all = []
freqs_all = []
accs = []

for i in range(4):
    mne_d = mne.read_epochs("../datasets/avi/single/filtered-4-14-sub-" + str(i) + ".fif", verbose=False)
    labels = []
    freqs = []

    for i in range(len(mne_d)):
        view = mne_d[i].compute_psd(method='multitaper', fmin=4, fmax=14, verbose=False)
        df = view.to_data_frame()
        freq = df['freq'][df['Oz'] == df['Oz'].max()].values[0]
        closest_freq = min(values, key=lambda x: abs(x - freq))
        freqs.append(closest_freq)
        labels.append(float(df['condition'][0]))

    labels_all.append(labels)
    freqs_all.append(freqs)
    accs.append(sum(np.array(freqs) == np.array(labels))/len(freqs))

print(accs)

[0.9523809523809523, 0.9523809523809523, 0.9523809523809523, 0.9523809523809523]


### Multi Target

In [6]:
warnings.filterwarnings('ignore')
# mne_d = mne.read_epochs("../datasets/avi/single/filtered-4-14-sub-0.fif")

# event_dict = {'6':0, '6.5':1, '7':2, '7.5':3, '8.2':4, '9.3':5, '10':6}
values = {6: 6, 6.5: 6.5, 7: 7, 7.5: 7.5, 8.2: 8.2, 9.3: 9.3, 10: 10}

labels_all = []
freqs_all = []
accs = []

for i in range(5):
    mne_d = mne.read_epochs("../datasets/avi/multi/filtered-4-14-sub-" + str(i) + ".fif", verbose=False)
    labels = []
    freqs = []

    for i in range(len(mne_d)):
        view = mne_d[i].compute_psd(method='multitaper', fmin=4, fmax=14, verbose=False)
        df = view.to_data_frame()
        freq = df['freq'][df['Oz'] == df['Oz'].max()].values[0]
        closest_freq = min(values, key=lambda x: abs(x - freq))
        freqs.append(closest_freq)
        labels.append(float(df['condition'][0]))

    labels_all.append(labels)
    freqs_all.append(freqs)
    accs.append(sum(np.array(freqs) == np.array(labels))/len(freqs))

print(accs)

[0.9523809523809523, 0.9523809523809523, 0.9523809523809523, 1.0, 1.0]
